In [219]:
%reset -f

In [220]:
from IPython.display import display, HTML
from sklearn import metrics
from sklearn import cross_validation
from scipy import interp
import pandas as pd
import numpy as np
import os

import sys
sys.path.insert(1,"../../src/")
from TypeFeatImputer import TypeFeatImputer
from i_score_parallel import i_score
from TypeFeatFS import DiscreteFS, ContinuousFS
from TypeFeatFilter import DiscreteFilter, ContinuousFilter
from OutlierFiltering import OutlierFiltering
from typeFeat_score import typeFeat_score

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.4f}'.format
plt.style.use('classic')

%matplotlib inline

In [238]:
ddf = []
pathDir = os.path.join('resources','results') 
path, dirs, files = os.walk(pathDir).next()
file_count = len(files)

for f in files:
    if "summary" not in str(f):
        ddf.append(pd.read_pickle(os.path.join('resources','results', str(f) )))
    
dfAll = pd.concat(ddf)
print dfAll.shape

(30, 27)


In [239]:
dfAll.cv_f1_mean = pd.to_numeric(dfAll.cv_f1_mean)
dfAll.cv_rec_mean = pd.to_numeric(dfAll.cv_rec_mean)
dfAll.cv_prec_mean = pd.to_numeric(dfAll.cv_prec_mean)

dfAll.cv_f1_std = pd.to_numeric(dfAll.cv_f1_std)
dfAll.cv_prec_std = pd.to_numeric(dfAll.cv_prec_std)
dfAll.cv_rec_std = pd.to_numeric(dfAll.cv_rec_std)
dfAll.exp = pd.to_numeric(dfAll.exp)

dfAll["cv_f1"] = dfAll["cv_f1_mean"].round(2).astype(str).str.cat(dfAll["cv_f1_std"].round(2).astype(str), sep="+/-")
dfAll["cv_prec"] = dfAll["cv_prec_mean"].round(2).astype(str).str.cat(dfAll["cv_prec_std"].round(2).astype(str), sep="+/-")
dfAll["cv_rec"] = dfAll["cv_rec_mean"].round(2).astype(str).str.cat(dfAll["cv_rec_std"].round(2).astype(str), sep="+/-")
dfAll["num_nones"] = np.sum(np.hstack(((dfAll.sm == "none").reshape(-1,1),(dfAll.fs == "none").reshape(-1,1))), axis=1)

print dfAll.columns.tolist()
print dfAll.shape

['exp', 'typeDisease', 'typeEncounter', 'typeHypothesis', 'typeDataFeatures', 'typeDiagnosis', 'size_tr', 'fs', 'sm', 'cls', 'metric', 'params', 'tr_f1', 'tr_prec', 'tr_rec', 'cv_f1_mean', 'cv_f1_std', 'cv_prec_mean', 'cv_prec_std', 'cv_rec_mean', 'cv_rec_std', 'test_f1', 'test_prec', 'test_rec', 'test_auc', 'time', 'pipeline', 'cv_f1', 'cv_prec', 'cv_rec', 'num_nones']
(30, 31)


/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  del sys.path[0]


In [240]:
dfAll.sort_values(["typeDisease","cv_f1_mean"], ascending=False,inplace=True)

print dfAll[dfAll['typeDataFeatures'] == "non_extended"].shape
dfAll[dfAll['typeDataFeatures'] == "non_extended"][["typeDataFeatures","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']]

(12, 31)


,typeDataFeatures,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,non_extended,f1_weighted,rf,"[gini, 12, 400]",0.6992,0.7027,0.6976,0.6153,0.6186,0.6134,0.6175,0.6150,0.6224,0.6101
0,non_extended,f1,rf,"[entropy, 10, 350]",0.6518,0.6573,0.6494,0.6133,0.6188,0.6107,0.6154,0.6125,0.6228,0.6104
0,non_extended,recall,rf,"[gini, 8, 350]",0.6294,0.6354,0.6268,0.6121,0.6179,0.6094,0.6131,0.6101,0.6202,0.6077
0,non_extended,f1_weighted,logReg,"[balanced, 0.005, l2]",0.6059,0.6106,0.6034,0.6027,0.6072,0.6003,0.6050,0.6025,0.6100,0.5971
0,non_extended,f1,logReg,"[balanced, 0.0001, l2]",0.5995,0.6071,0.5964,0.5941,0.6021,0.5909,0.5984,0.5952,0.6061,0.5930
0,non_extended,recall,logReg,"[balanced, 1e-05, l2]",0.5883,0.5996,0.5848,0.5854,0.5968,0.5819,0.5906,0.5871,0.6021,0.5884
0,non_extended,f1_weighted,knn,"[uniform, 11]",0.6633,0.6747,0.6786,0.5817,0.5864,0.6025,0.5839,0.6037,0.5881,0.5633
0,non_extended,recall,knn,"[uniform, 1]",0.9999,0.9999,0.9999,0.5530,0.5518,0.5544,0.5594,0.5612,0.5581,0.5422
0,non_extended,f1,knn,"[uniform, 1]",0.9999,0.9999,0.9999,0.5530,0.5518,0.5544,0.5594,0.5612,0.5581,0.5422
0,non_extended,recall,nb,[],0.2380,0.7035,0.4086,0.2381,0.6797,0.4085,0.2370,0.4080,0.5970,0.5002


In [235]:
dfAll[dfAll['typeDataFeatures'] == "extended"][["typeDataFeatures","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']]

,typeDataFeatures,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,extended,f1_weighted,rf,"[gini, 12, 200]",0.6985,0.7020,0.6968,0.6169,0.6203,0.6149,0.6140,0.6114,0.6192,0.6067
0,extended,f1,rf,"[entropy, 10, 350]",0.6522,0.6576,0.6499,0.6138,0.6190,0.6113,0.6144,0.6116,0.6210,0.6085
0,extended,recall,rf,"[entropy, 8, 250]",0.6289,0.6355,0.6261,0.6107,0.6169,0.6080,0.6123,0.6092,0.6200,0.6075
0,extended,f1_weighted,logReg,"[balanced, 0.01, l2]",0.6088,0.6134,0.6064,0.6054,0.6096,0.6031,0.6074,0.6050,0.6121,0.5993
0,extended,f1,logReg,"[balanced, 0.0001, l2]",0.6006,0.6080,0.5976,0.5961,0.6038,0.5930,0.5993,0.5962,0.6072,0.5940
0,extended,recall,logReg,"[balanced, 1e-05, l2]",0.5885,0.5994,0.5850,0.5853,0.5961,0.5818,0.5909,0.5874,0.6021,0.5884
0,extended,f1_weighted,knn,"[distance, 11]",0.9999,0.9999,0.9999,0.5782,0.5832,0.6001,0.5832,0.6044,0.5884,0.5628
0,extended,recall,knn,"[uniform, 1]",0.9999,0.9999,0.9999,0.5523,0.5508,0.5541,0.5553,0.5573,0.5538,0.5377
0,extended,f1,knn,"[uniform, 1]",0.9999,0.9999,0.9999,0.5523,0.5508,0.5541,0.5553,0.5573,0.5538,0.5377
0,extended,f1,nb,[],0.2380,0.7035,0.4086,0.2381,0.6797,0.4085,0.2370,0.4080,0.5970,0.5002


In [241]:
dfAll[dfAll['typeDataFeatures'] == "extended_extra"][["typeDataFeatures","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']]

,typeDataFeatures,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,extended_extra,recall,rf,"[entropy, 10, 500]",0.6501,0.6513,0.6491,0.6153,0.6164,0.6144,0.6136,0.6123,0.6156,0.6027
0,extended_extra,f1,rf,"[entropy, 10, 500]",0.6502,0.6515,0.6493,0.6152,0.6164,0.6143,0.6137,0.6124,0.6157,0.6028
0,extended_extra,f1_weighted,rf,"[gini, 12, 150]",0.6996,0.6999,0.6993,0.6144,0.6150,0.6138,0.6156,0.6143,0.6173,0.6044
0,extended_extra,f1_weighted,logReg,"[balanced, 1, l1]",0.6109,0.6152,0.6086,0.6081,0.6120,0.6059,0.6065,0.6041,0.6110,0.5981
0,extended_extra,recall,logReg,"[balanced, 0.01, l1]",0.6091,0.6140,0.6066,0.6041,0.6089,0.6017,0.6046,0.6019,0.6101,0.5972
0,extended_extra,f1,logReg,"[balanced, 0.01, l1]",0.6091,0.6140,0.6066,0.6041,0.6089,0.6017,0.6046,0.6019,0.6101,0.5972


In [189]:
dfAll.sort_values(["typeDisease","cv_f1_mean"], ascending=False,inplace=True)
dfAll.groupby(["typeDisease"])[["size_tr","sm","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']].first()

,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
typeDisease,,,,,,,,,,,,,,,
subset,1.00,none,f1_weighted,rf,"[gini, 12, 200]",0.70,0.70,0.70,0.62,0.62,0.61,0.61,0.61,0.62,0.61


In [161]:
dfAll[dfAll["typeDisease"] == "Neoplasms_1"][["size_tr","sm","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']]

,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,0.40,none,f1_weighted,rf,"[gini, 6, balanced, 100]",0.83,0.83,0.83,0.64,0.63,0.65,0.62,0.63,0.62,0.54
0,0.20,none,f1_weighted,logReg,"[None, 0.0001, l2]",0.66,0.67,0.71,0.64,0.65,0.69,0.63,0.68,0.63,0.54
0,0.60,none,f1_weighted,rf,"[gini, 6, balanced, 250]",0.82,0.82,0.82,0.64,0.63,0.64,0.62,0.62,0.62,0.55
0,0.40,none,recall,rf,"[entropy, 4, balanced, 250]",0.74,0.75,0.74,0.63,0.64,0.62,0.61,0.61,0.62,0.55
0,0.20,none,roc_auc,logReg,"[None, 0.005, l2]",0.64,0.67,0.71,0.63,0.64,0.69,0.63,0.69,0.63,0.53
0,0.40,none,roc_auc,nb,[],0.63,0.63,0.68,0.62,0.61,0.66,0.63,0.67,0.62,0.54
0,0.40,none,recall,nb,[],0.63,0.63,0.68,0.62,0.61,0.66,0.63,0.67,0.62,0.54
0,0.40,none,f1_weighted,nb,[],0.63,0.63,0.68,0.62,0.61,0.66,0.63,0.67,0.62,0.54
0,0.20,none,f1_weighted,rf,"[gini, 6, balanced, 200]",0.88,0.88,0.88,0.62,0.61,0.62,0.61,0.61,0.60,0.53
0,0.60,none,f1_weighted,logReg,"[balanced, 0.05, l2]",0.63,0.67,0.61,0.61,0.65,0.59,0.57,0.55,0.61,0.54


In [197]:
#### Save results

In [211]:
import time
dfAll.iloc[:,:-4].to_pickle(os.path.join(pathDir, "summary" + time.strftime("%Y%m%d-%H%M%S") + ".pkl"))

In [213]:
dd = pd.read_pickle(os.path.join(pathDir,"summary20171017-171147.pkl"))
print dd.columns

Index([u'exp', u'typeDisease', u'typeEncounter', u'typeHypothesis',
       u'typeDataFeatures', u'typeDiagnosis', u'size_tr', u'fs', u'sm', u'cls',
       u'metric', u'params', u'tr_f1', u'tr_prec', u'tr_rec', u'cv_f1_mean',
       u'cv_f1_std', u'cv_prec_mean', u'cv_prec_std', u'cv_rec_mean',
       u'cv_rec_std', u'test_f1', u'test_prec', u'test_rec', u'test_auc',
       u'time', u'pipeline'],
      dtype='object')
